In [1]:
%matplotlib inline
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json
import geopandas as gpd
from ipywidgets import interact, fixed, HTML, widgets
from IPython.display import display

In [2]:
def showMap(point):
    url = 'https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson'
    world_layer = gpd.read_file(url)
    world_layer = world_layer.plot(figsize=(15,15),facecolor='none', edgecolor='black')
    point.plot(ax=world_layer, markersize=20, color='red')
    plt.show()    

def getCountyData(name='Algiers', df=None):
    url = 'https://www.google.com.tw/maps/place/' + name
 
    html = widgets.HTML(
        value='<a target="_blank" href='+ url +'>Go to Google Map</a>'
    )
    display(html)
    display(df.loc[df['name'] == name, ['capital', 'name', 'region', 'population', 'area', 'timezones', 'latlng']])
    
    df = df.loc[df['name'] == name, ['capital', 'name', 'region', 'population', 'area', 'timezones', 'latlng']]
    
    name = df['name'].tolist()
    capital = df['capital'].tolist()
    region = df['region'].tolist()    
    population = df['population'].tolist()
    area = df['area'].tolist()
    timezones = df['timezones'].tolist()
    latlng = df['latlng'].tolist()
    lat = [i[0] for i in latlng]
    lng = [i[1] for i in latlng]
    
    df = pd.DataFrame({
        'name': name,
        'capital': capital,
        'region': region,
        'population': population,
        'area': area,
        'timezones': timezones,
        'lat': lat,
        'lng': lng
    })

    df['Coordinates'] = list(zip(df.lng, df.lat))
    df['Coordinates'] = df['Coordinates'].apply(Point)
    county_point = gpd.GeoDataFrame(df, geometry='Coordinates')
    county_point.crs = {'init' :'epsg:4326'}

    showMap(county_point)
    
def getRegionData(region):
    url = 'https://restcountries.eu/rest/v2/region/' + region + \
    '?fields=name;capital;region;latlng;population;area;timezones'
    r = requests.get(url)
    result = r.json()
    df = pd.DataFrame(result)
    county_name = df['name'].tolist()
    interact(getCountyData, name=county_name, df=fixed(df))

interact(getRegionData, region=['Africa', 'Americas', 'Asia', 'Europe', 'Oceania'])

interactive(children=(Dropdown(description='region', options=('Africa', 'Americas', 'Asia', 'Europe', 'Oceania…

<function __main__.getRegionData(region)>

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from ipywidgets import interact,widgets,fixed
from IPython.display import display
import matplotlib as mpl
mpl.rc('font', family='Noto Sans CJK TC')

In [13]:
TW = gpd.read_file('台灣TWD97鄉鎮市\TOWN_MOI_1071226.shp',encoding = 'utf-8')

df = pd.read_excel('村里學歷人數改.xlsx',encoding = 'utf-8')
colnames = df.loc[0].values
df = df[2:]
df.columns = colnames
df.index = range(1,7760)

df2 = df.groupby(by='TOWNNAME').sum()

df2.drop(['統計年','區域別代碼','區域別','COUNTYNAME','村里名稱'],axis = 1,inplace = True)
df2.reset_index(inplace=True)

df3 = pd.merge(TW, df2,how='left')
df3.drop(['TOWNID','TOWNCODE','TOWNENG','COUNTYID','COUNTYCODE'],axis = 1,inplace = True)
df3.crs= {'init' :'epsg:3826'}

df3 = df3.fillna(0)

edu = pd.DataFrame(df3.loc[:,'博士畢':'不識字'])

def ShowMeTheTown(name='信義區'):
    global df3
    display(df3.loc[df3['TOWNNAME']==name])
    edu.loc[10].plot.bar(figsize = (5,5))
def ShowMeTheCounty(County='臺北市'):
    global df3
    df3[df3.COUNTYNAME.str.contains(County)]
    df3.loc[df3['COUNTYNAME']==County].plot(figsize = (10,10))
    interact(ShowMeTheTown,name=df3['TOWNNAME'],df=fixed(df3))

interact(ShowMeTheCounty,County =['南投縣', '嘉義市', '嘉義縣', '基隆市', '宜蘭縣', '屏東縣', '彰化縣', '新北市', '新竹市', '新竹縣','桃園市', '澎湖縣', '臺中市', '臺北市', '臺南市', '臺東縣', '花蓮縣', '苗栗縣', '連江縣', '金門縣','雲林縣', '高雄市'])

interactive(children=(Dropdown(description='County', index=13, options=('南投縣', '嘉義市', '嘉義縣', '基隆市', '宜蘭縣', '屏東…

<function __main__.ShowMeTheCounty(County='臺北市')>